# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name(s): Edmond Pan
###### Python version: 3.6

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the seven functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [20]:
# Additional libraries used in the Naive Bayes implementation
import numpy as np
import math as mth
from collections import defaultdict

In [21]:
class DataSet:

    def __init__(self, filename):
        """
        Reads in the csv file into an appropriate data structure
        :param filename: Name of the .csv file
        """
        # Variables to store metadata about the table structure
        self.num_rows = 0
        self.num_cols = 0
        self.table = []
        file = open('2018S1-proj1_data/' + filename, 'r')
        for line in file.readlines():
            # Split based on common to get the values
            row = line.split(',')
            self.num_cols = len(row)
            # Add row to table and increment row count
            self.table.append(row)
            self.num_rows += 1
        file.close()

    def get_num_rows(self):
        """
        Gets the number of rows in the table
        :return: Integer referencing number of rows in the table
        """
        return self.num_rows

    def get_num_cols(self):
        """
        Gets the number of cols in the table
        :return: Integer referring to number of cols in table
        """
        return self.num_cols

    def get_table(self):
        """
        Gets the stored table
        :return: Returns a list of rows
        """
        return self.table

    def random_initial(self):
        """
        Function that replaces the class labels with random (non-uniform)
        class distributions. NOTE ONLY USE WHEN DOING UNSUPERVISED NB
        :return: None
        """
        # Default dict containing all possible classes
        classes = defaultdict(float)
        for row in self.table:
            # Keep them all at 0 since they will b replaced with
            # random fractional counts that sum to 1
            classes[row[-1]] = 0

        # Now remove the class labels and add the classes dictionaries while
        # initialising the values to random fractional counts
        for row in self.table:
            # Add the random values to the dictionary
            values = np.random.dirichlet(np.ones(len(classes)), size=1)
            i = 0
            for key, value in classes.items():
                classes[key] = values[0][i]
                i += 1
            # Replace the old class label with the random class distribution.
            # Make sure to return a copy of classes instead of passing the reference
            row[-1] = classes.copy()
        return
    
def print_confusion(predicted, expected):
    """
    Function to print the confusion matrix from a list of predicted and
    expected values
    :param predicted: A list of predicted values 
    :param expected:  A list of expected values
    :return: None
    """
    if len(predicted) != len(expected):
        print("FATAL ERROR: List lengths do not match")
        return
    # Structure to store the matrix data. Can access the individual values
    # in this format matrix[predicted_class][expected_class]
    matrix = defaultdict(lambda: defaultdict(int))
    for i in range(len(predicted)):
        matrix[predicted[i]][expected[i]] += 1
    return matrix



In [22]:
def preprocess(filename):
    """
    Function that builds a DataSet object from a .csv file containing data
    :param filename: A string containing the name of the .csv file
    :return: A DataSet object of the .csv file
    """
    return DataSet(filename)


In [23]:
class SupervisedModel:

    def __init__(self, dataset):
        """
        Constructor for a SupervisedModel object
        :param dataset: A DataSet object containing the .csv file to calculate
                        probabilities from
        """
        # Variables to store counts to use in calculating prior and posterior probabilities
        self.prior_counts, self.posterior_counts, self.missing_counts = self.create_counts(dataset)

        self.prior_prob, self.posterior_prob = self.__calc_probabilities__(dataset)

    def get_prior_counts(self):
        return self.prior_counts

    def get_posterior_counts(self):
        return self.posterior_counts

    def get_prior_prob(self):
        return self.prior_prob

    def get_posterior_prob(self):
        return self.posterior_prob

    @classmethod
    def create_counts(cls, dataset):
        """
        Function to count up the frequencies of the different classes and attributes
        in the dataset
        :param dataset: A DataSet object to read and produce counts from
        :return: 2 dictionaries in tuple form. i.e. (dict1, dict2) that represent the
                 counts to be used for prior and posterior probabilities respectively
        """
        prior_counts = defaultdict(int)
        # Triple nested dictionary. Key of first dict contains attribute names, second dict
        # contains attribute values as keys, third dict contains class names as keys
        posterior_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        # Structure to store how many missing values exist for each attribute. A key refers
        # to the attribute name, second key refers to class name
        missing_counts = defaultdict(lambda: defaultdict(int))

        # Add prior counts to default dict
        for row in dataset.table:
            # Assuming last column is the class
            prior_counts[row[-1]] += 1
        # Add posterior counts to data structure
        for row in dataset.table:
            row_index = 0
            for attribute in row[:-1]:
                # If attribute == ?, then add to missing counts dict
                if attribute == '?':
                    missing_counts[row_index][row[-1]] += 1
                else:
                    # Initialise all the dictionaries of each possible attribute value
                    # to contain all possible classes. Note missing values will not contribute
                    # to the counts and are treated as a separate count that will not be used
                    for key, value in prior_counts.items():
                        posterior_counts[row_index][attribute][key]
                row_index += 1
            # Now add the counts
            row_index = 0
            for attribute in row[:-1]:
                # Skip adding missing values
                if attribute == '?':
                    row_index += 1
                else:
                    # Add to the counts
                    posterior_counts[row_index][attribute][row[-1]] += 1
                    row_index += 1

        return prior_counts, posterior_counts, missing_counts

    def __calc_probabilities__(self, dataset):
        """
        Function that takes the counts and produces prior and posterior probabilities
        :param dataset: A DataSet object to create the probabilities from
        :return: 2 dictionaries in tuple form. i.e. (dict1, dict2) that represent prior
                 and posterior probabilities respectively
        """
        prior_prob = defaultdict(float)
        # Format for this dict is: First dict key refers to attribute name, second dict key refers
        # attribute value and third dict key refers to class name. E.g. P[0]['20-29']['recurrence-events\n']
        posterior_prob = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
        # Values for laplace smoothing
        k = 1

        # Calculating the prior probabilities
        for key, value in self.prior_counts.items():
            prior_prob[key] = value/dataset.get_num_rows()

        # Calculating the posterior probabilities
        for attr_name, val_dict in self.posterior_counts.items():
            # Value to add to denominator when doing Laplace smoothing.
            unique_attr_num = len(self.posterior_counts[attr_name].items())
            for attr_val, class_dict in self.posterior_counts[attr_name].items():
                for class_name, count in self.posterior_counts[attr_name][attr_val].items():
                    # Do Laplace smoothing of the counts
                    numerator = count + k
                    denominator = self.prior_counts[class_name] + unique_attr_num - \
                                  self.missing_counts[attr_name][class_name]
                    posterior_prob[attr_name][attr_val][class_name] = numerator/denominator

        return prior_prob, posterior_prob

    
def train_supervised(data):
    """
    Function that builds a supervised NB model
    :param data: A DataSet object to build a supervised NB model from
    :return: A SupervisedModel object
    """
    return SupervisedModel(data)

In [24]:
def arg_max(dictionary):
    """
    Function that returns the key that has the highest value in a dictionary
    :param dictionary: Dictionary containing the values to check
    :return: A string containing the name of key with the highest value
    """
    return max(dictionary, key=lambda key: dictionary[key])

def predict_sup_single(data_instance, model):
    """
    This function takes a single instance and returns a classification
    :param data_instance: A list of attribute values
    :param model: A SupervisedModel object to use for evaluation
    :return: A string corresponding to the most likely class this instance belongs to
    """
    nb_scores = defaultdict(float)
    for class_name, value in model.get_prior_counts().items():
        prior = model.get_prior_prob()
        posterior = model.get_posterior_prob()
        nb_scores[class_name] = prior[class_name]
        attr_index = 0
        for attribute in data_instance:
            # If test instance has missing value, skip it
            if attribute == '?':
                attr_index += 1
            else:
                nb_scores[class_name] *= posterior[attr_index][attribute][class_name]
                attr_index += 1
    return arg_max(nb_scores)


def log_pred_sup_single(data_instance, model):
    """
    This function takes a single instance and returns a classification
    :param data_instance: A list of attribute values
    :param model: A SupervisedModel object to use for evaluation
    :return: A string corresponding to the most likely class this instance belongs to
    """
    nb_scores = defaultdict(float)
    for class_name, value in model.get_prior_counts().items():
        prior = model.get_prior_prob()
        posterior = model.get_posterior_prob()
        nb_scores[class_name] = mth.log(prior[class_name])
        attr_index = 0
        for attribute in data_instance:
            if attribute == '?':
                attr_index += 1
            else:
                nb_scores[class_name] += mth.log(posterior[attr_index][attribute][class_name])
                attr_index += 1
    return arg_max(nb_scores)


def predict_supervised(filename):
    """
    Function that predicts the class for a set of instances
    :param filename: The filename of the .csv file to create predictions from
    :return: A list of predicted classes with indices corresponding to the row number
    """
    predicted = []
    # Build the model from the dataset.
    data = DataSet(filename)
    model = SupervisedModel(data)
    for row in data.table:
        # Also skip last attribute as that is the class distribution
        classified = predict_sup_single(row[:-1], model)
        if classified == log_pred_sup_single(row[:-1], model):
            predicted.append(classified)
    return predicted


In [25]:
def evaluate_supervised(filename):
    """
    Function that prints the accuracy rating for a given set of predictions and
    constructs a confusion matrix
    :param filename: The name of the .csv files the predictions were created from
    :return: A confusion matrix in the format of a 2D dictionary accessible in the
             format matrix[predicted_class][expected_class]
    """
    # Get the expected classes from the dataset
    dataset = DataSet(filename)
    expected = []
    for row in dataset.table:
        expected.append(row[-1])
    predicted = predict_supervised(filename)
    # Now for each instance check if it matches the expected
    num_correct = 0
    total_instances = len(expected)
    curr_inst = 0
    for pred in predicted:
        if pred == expected[curr_inst]:
            num_correct += 1
        curr_inst += 1
    print('Accuracy = ' + str((num_correct/total_instances) * 100))
    return print_confusion(predicted, expected)

In [26]:
class UnsupervisedModel:

    def __init__(self, dataset):
        """
        Constructor for an UnsupervisedModel
        :param dataset: A DataSet object containing the dataset to classify
        """
        # Variables to store prior counts and posterior counts
        self.prior_counts, self.posterior_counts, self.missing_counts = self.create_counts(dataset)
        # Variables to store prior and posterior probabilities
        self.prior_prob, self.posterior_prob = self.__calc_probabilities__(dataset)

    def iterate(self, dataset, n=3):
        """
        Function that iteratively assigns new class distributions to dataset calculated
        from the current model
        :param dataset: A DataSet object to iteratively assign new class distributions
        :param n: The number of iterations to perform. Defaults to 3
        :return: None
        """
        for i in range(n):
            self.recalculate(dataset)
        return

    @classmethod
    def recalculate(cls, dataset):
        """
        Function that will recalculate all of the class distributions in
        dataset and assign more reliable distributions
        :param dataset: A DataSet object to be altered
        :return: None
        """
        # Contains the current model of the dataset
        old_model = UnsupervisedModel(dataset)
        # Go through every instance of the dataset and reassign the class distributions
        for row in dataset.table:
            row[-1] = predict_uns_single(row[:-1], old_model)
        return

    @classmethod
    def create_counts(cls, dataset):
        """
        Function to produce the fractional counts of the different classes and attributes
        in the dataset
        :param dataset: A DataSet object to read and produce counts from
        :return: 2 dictionaries in tuple form. i.e. (dict1, dict2) that represent the
                 counts to be used for prior and posterior probabilities respectively
        """
        prior_counts = defaultdict(float)
        # Triple nested dictionary. Key of first dict contains attribute names, second dict
        # contains attribute values as keys, third dict contains class names as keys
        posterior_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
        # Structure to store how many missing values exist for each attribute. A key refers
        # to the attribute name, second key refers to class name
        missing_counts = defaultdict(lambda: defaultdict(float))

        # Add prior counts to default dict
        for row in dataset.table:
            # Add the corresponding fractional count to the correct class
            for class_name, value in row[-1].items():
                prior_counts[class_name] += value

        for row in dataset.table:
            row_index = 0
            for attribute in row[:-1]:
                # If attribute == ?, then add to missing counts dict
                if attribute == '?':
                    for class_name, value in row[-1].items():
                        missing_counts[row_index][class_name] += value
                else:
                    # Initialise all the dictionaries of each possible attribute value
                    # to contain all possible classes. Note missing values will not contribute
                    # to the counts and are treated as a separate count that will not be used
                    for key, value in prior_counts.items():
                        posterior_counts[row_index][attribute][key]
                row_index += 1
            # Now add the counts
            row_index = 0
            for attribute in row[:-1]:
                # Skip adding missing values
                if attribute == '?':
                    row_index += 1
                else:
                    # Add to the counts
                    for class_name, value in row[-1].items():
                        posterior_counts[row_index][attribute][class_name] += value
                    row_index += 1

        return prior_counts, posterior_counts, missing_counts

    def __calc_probabilities__(self, dataset):
        """
        Function that takes the counts and produces prior and posterior probabilities
        :param dataset: A DataSet object to create the probabilities from
        :return: 2 dictionaries in tuple form. i.e. (dict1, dict2) that represent prior
                 and posterior probabilities respectively
        """
        prior_prob = defaultdict(float)
        # Format for this dict is: First dict key refers to attribute name, second dict key refers
        # attribute value and third dict key refers to class name. E.g. P[0]['20-29']['recurrence-events\n']
        posterior_prob = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))

        # Calculating the prior probabilities by dividing by the number of instances
        for class_name, value in self.prior_counts.items():
            prior_prob[class_name] = value/dataset.get_num_rows()

        # Now calculate the posterior probabilities by dividing the fractional counts
        # by the fractional class counts
        for attr_name, val_dict in self.posterior_counts.items():
            for attr_val, class_dict in self.posterior_counts[attr_name].items():
                for class_name, count in self.posterior_counts[attr_name][attr_val].items():
                    numerator = count
                    # Subtract the fractional counts from the total if this attribute of
                    # the instance is missing
                    denominator = self.prior_counts[class_name] - self.missing_counts[attr_name][class_name]
                    posterior_prob[attr_name][attr_val][class_name] = numerator/denominator

        return prior_prob, posterior_prob
    
    
def train_unsupervised(data):
    """
    Function that builds an unsupervised NB model
    :param data: A DataSet object to build the model the unsupervised NB model from
    :return: An UnsupervisedModel object
    """
    return UnsupervisedModel(data)

In [27]:
def predict_uns_single(data_instance, model):
    """
    This function uses a trained unsupervised model to predict the class distribution
    for test instance
    :param data_instance: A list of attributes to predict the class distribution for 
    :param model: An UnsupervisedModel object that will be used to make the predictions
    :return: A dictionary containing a new class distribution for the test_instance
    """
    class_dist = defaultdict(float)
    for class_name, value in model.prior_counts.items():
        prior = model.prior_prob
        posterior = model.posterior_prob
        class_dist[class_name] = prior[class_name]
        attr_index = 0
        for attribute in data_instance:
            # If the data instance has a missing value, skip it during
            # calculations
            if attribute == '?':
                attr_index += 1
            else:
                class_dist[class_name] *= posterior[attr_index][attribute][class_name]
                attr_index += 1
    # Now normalise the values to get the new class distribution.
    # Create a copy to allow the original class_dist to be modified
    temp_values = class_dist.copy()
    for class_name, value in temp_values.items():
        class_dist[class_name] = value/sum(temp_values.values())

    return class_dist


def predict_unsupervised(filename, n=3):
    """
    Function that predicts the class for a set of instances
    :param filename: The filename of the .csv file that contains the set of instances
                     to make predictions from
    :param n: The number of iterations to perform. Defaults to 3
    :return: A list of predicted classes with indices corresponding to the row number
    """
    predicted = []
    # Read and build the model from the dataset.
    data = DataSet(filename)
    data.random_initial()
    model = UnsupervisedModel(data)
    # Iterate over the specified amount of times
    model.iterate(data, n)
    model = UnsupervisedModel(data)
    for row in data.table:
        # Also skip last attribute as that is the class distribution
        predicted.append(arg_max(predict_uns_single(row[:-1], model)))
    return predicted


In [28]:
def evaluate_unsupervised(filename, n=3):
    """
    Function that prints the accuracy rating for a given dataset when using Naive Bayes
    to classify it's instances. NOTE: It uses all instances in the dataset to train and
    will also be testing on the same instances.
    :param filename: filename of the dataset to test on
    :param n: number of times to iterate when building the model. Default is 3
    :return: A confusion matrix in the format of a 2D dictionary accessible in the
             format matrix[predicted_class][expected_class]
    """
    predicted = predict_unsupervised(filename, n)
    expected = []
    data = DataSet(filename)
    for row in data.table:
        expected.append(row[-1])
    matrix = print_confusion(predicted, expected)
    accuracy = 0
    total_instances = data.get_num_rows()
    for key, value in matrix.items():
        accuracy += max(matrix[key].values())
    print('Accuracy = ' + str((accuracy/total_instances) * 100))
    return matrix

Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.

## Question 1 Answer

Put answer to question 1 here

## Question 2 Answer

Put answer to question 2 here

In [29]:
sup_ans = evaluate_supervised('mushroom.csv')

Accuracy = 95.88872476612507


In [35]:
uns_ans = evaluate_unsupervised('flu-test.csv')

Accuracy = 80.0
